In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
%matplotlib inline
import seaborn as sns

from sklearn.preprocessing import MultiLabelBinarizer

In [33]:
all_cars = pd.read_csv('dataset_V2.csv')
all_cars.head(3)

,Unnamed: 0,bodyType,brand,car_url,color,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,ПТС,Привод,Руль,Состояние,Таможня,price
0,0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,белый,Машина в отличном состоянии🥇\nКомплектация Ele...,1.8 LTR,152 N12,"{'cruise-control': True, 'tinted-glass': True,...",бензин,...,LIFTBACK AUTOMATIC 1.8,автоматическая,EUROPEAN,2.0,DUPLICATE,FORWARD_CONTROL,LEFT,True,True,829000.0
1,1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,бежевый,Хорошее техническое и косметическое\nсостояние...,1.8 LTR,152 N12,"{'cruise-control': True, 'esp': True, 'adaptiv...",бензин,...,LIFTBACK AUTOMATIC 1.8,автоматическая,EUROPEAN,3.0,ORIGINAL,FORWARD_CONTROL,LEFT,True,True,785000.0
2,2,внедорожник 5 дв.,SKODA,https://auto.ru/cars/used/sale/skoda/yeti/1105...,пурпурный,Идеальное состояние автомобиля\nДилер Краснояр...,1.2 LTR,105 N12,"{'computer': True, 'ptf': True, 'rain-sensor':...",бензин,...,ALLROAD_5_DOORS ROBOT 1.2,роботизированная,EUROPEAN,1.0,ORIGINAL,FORWARD_CONTROL,LEFT,True,True,745000.0


In [34]:
all_cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104663 entries, 0 to 104662
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Unnamed: 0            104663 non-null  int64  
 1   bodyType              104450 non-null  object 
 2   brand                 104451 non-null  object 
 3   car_url               104663 non-null  object 
 4   color                 104451 non-null  object 
 5   description           104451 non-null  object 
 6   engineDisplacement    104451 non-null  object 
 7   enginePower           104451 non-null  object 
 8   equipment_dict        104449 non-null  object 
 9   fuelType              104449 non-null  object 
 10  mileage               104449 non-null  float64
 11  modelDate             104450 non-null  float64
 12  model_info            97958 non-null   object 
 13  model_name            104449 non-null  object 
 14  numberOfDoors         104450 non-null  float64
 15  

датасет доставточно "грязный", проверим на наличие пропусков, уберем лишние данные и преобразуем имеющиеся в формат, пригодный для анализа

In [35]:
all_cars.isnull().sum()

Unnamed: 0                 0
bodyType                 213
brand                    212
car_url                    0
color                    212
description              212
engineDisplacement       212
enginePower              212
equipment_dict           214
fuelType                 214
mileage                  214
modelDate                213
model_info              6705
model_name               214
numberOfDoors            213
productionDate           212
super_gen                215
vehicleConfiguration     213
vehicleTransmission      214
vendor                  6704
Владельцы                221
ПТС                      216
Привод                   216
Руль                     214
Состояние                233
Таможня                  223
price                   8750
dtype: int64

пропуски есть практически в каждой колонке, но их сравнительно незначительнное количетсво. Поэтому спокойно их удаляем

In [37]:
all_cars = all_cars.dropna()
all_cars.isnull().sum()

Unnamed: 0              0
bodyType                0
brand                   0
car_url                 0
color                   0
description             0
engineDisplacement      0
enginePower             0
equipment_dict          0
fuelType                0
mileage                 0
modelDate               0
model_info              0
model_name              0
numberOfDoors           0
productionDate          0
super_gen               0
vehicleConfiguration    0
vehicleTransmission     0
vendor                  0
Владельцы               0
ПТС                     0
Привод                  0
Руль                    0
Состояние               0
Таможня                 0
price                   0
dtype: int64

In [38]:
# признак 'car_url' будем ипользовать в качестве идентификатора, извдечем оттуда id объявления
all_cars['car_url'][5]

'https://auto.ru/cars/used/sale/skoda/octavia/1105846102-35605d51/'

In [39]:
all_cars.url_split = all_cars.car_url.str.split('/')
all_cars.url_split[15648]

<ipython-input-39-a395f9173830>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  all_cars.url_split = all_cars.car_url.str.split('/')


['https:',
 '',
 'auto.ru',
 'cars',
 'used',
 'sale',
 'honda',
 'accord',
 '1105508916-b03d1806',
 '']

In [40]:
id_car = [power[0:][8] for power in all_cars.url_split]
id_car_split = [x.split('-') for x in id_car]
all_cars['id_car'] = [q[0] for q in id_car_split]
all_cars['id_car'] = pd.to_numeric(all_cars['id_car'])
all_cars['id_car']

0         1105765221
1         1105672391
2         1105201873
3         1105607078
4         1105561522
             ...    
104656    1105731924
104658    1104571613
104659    1105831300
104660    1105408230
104661    1105396628
Name: id_car, Length: 95876, dtype: int64

In [41]:
all_cars['id_car'].duplicated().value_counts()

False    95114
True       762
Name: id_car, dtype: int64

в id объявлений встречаются дубликаты, но их небольшое количетсво, удаляем их

In [42]:
all_cars = all_cars.drop_duplicates(subset='id_car', keep=False, inplace=False)

рассмотрим признак super_gen, где собраны характеристики автомобиля. бОльшая часть дублируется в других признаках, но здесь есть информация о расходе топлива, что может быть полезной для построения модели. Выделим его в отдельный признак 

In [46]:
all_cars[['id_car', 'super_gen']].loc[12][1]

"{'id': '5040501', 'name': 'Scout', 'nameplate': 'Scout', 'displacement': 1798, 'engine_type': 'GASOLINE', 'gear_type': 'ALL_WHEEL_DRIVE', 'transmission': 'MECHANICAL', 'power': 152, 'power_kvt': 112, 'human_name': 'Scout 1.8 MT (152 л.с.) 4WD', 'acceleration': 8.7, 'clearance_min': 179, 'fuel_rate': 7.8}"

In [47]:
all_cars['fuel_rate'] = [rate[-18:] for rate in all_cars['super_gen'] ]
all_cars['fuel_rate'] = all_cars['fuel_rate'].str.replace(('}'), '')
all_cars['fuel_rate'] = all_cars['fuel_rate'].str.replace(("'"), '')
all_cars['fuel_rate'] = all_cars['fuel_rate'].str.replace(("'"), '')
all_cars['fuel_rate'].value_counts(ascending = True).index[:20]

Index(['2, fuel_rate: 4', 'earance_min: 186', 'fuel_rate: 16.4',
       ' fuel_rate: 2.4', 'earance_max: 160', '3.3 MT (75 л.с.)',
       'earance_min: 254', 'ectric_range: 50', 'ery_capacity: 17',
       'earance_min: 131', 'earance_min: 163', 'earance_max: 184',
       'ery_capacity: 38', '6, fuel_rate: 4', 'fuel_rate: 19.4',
       'fuel_rate: 19.2', 'fuel_rate: 17.3', 'earance_min: 290',
       'y_capacity: 41.4', ' fuel_rate: 3.1'],
      dtype='object')

как видно, есть данные, которые попали в список случайно. обработаем их

In [48]:
fuel = []
for f  in all_cars['fuel_rate']:
    if 'fuel_rate:' in f:
        fuel.append(f)
    elif 'fuel_rate:' not in f:
        fuel.append('w') 
        """ вместо пропусков сначала проставим случайное значение,  пусть будет буква w """
        
all_cars['contain2'] = fuel
all_cars['contain2_split'] = all_cars['contain2'].str.split(':')
all_cars['contain2_split']

0         [ fuel_rate,  7.7]
1         [ fuel_rate,  7.7]
2         [ fuel_rate,  6.4]
3         [ fuel_rate,  6.3]
4         [ fuel_rate,  7.4]
                 ...        
104656    [ fuel_rate,  6.8]
104658    [fuel_rate,  14.4]
104659    [ fuel_rate,  7.8]
104660    [ fuel_rate,  9.5]
104661                   [w]
Name: contain2_split, Length: 94352, dtype: object

In [49]:
rate2 = []
for rate in all_cars['contain2_split']:
    if len(rate)> 1:
        rate2.append(rate[1])
    else:
        rate2.append(rate[0])
        
all_cars['rate2'] = rate2
all_cars['rate2'].value_counts()

w        6653
 7.4     2430
 6.7     2293
 7.5     2208
 7.9     2201
         ... 
 16.2       1
 19.2       1
 17.3       1
 19.4       1
 3.1        1
Name: rate2, Length: 163, dtype: int64

сначала найдем медиану и среднее списка не учитывая пропуски

In [50]:
rate_w = all_cars[~all_cars.rate2.str.match("w", na=False)]
rate_w['rate2'] = pd.to_numeric(rate_w['rate2'])

<ipython-input-50-31dfc8d30bfc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rate_w['rate2'] = pd.to_numeric(rate_w['rate2'])


In [51]:
print (rate_w['rate2'].median())
print(rate_w['rate2'].mean())
print (rate_w['rate2'].mode())

7.8
8.208782312227324
0    7.4
dtype: float64


In [52]:
# заменим пропуски, которые мы ранее обозначили через 'w' медианой признака, и преобразуем данные в числовой формат
all_cars['rate2'] = all_cars['rate2'].str.replace(('w'), '7.8')
all_cars['rate_fuel'] = pd.to_numeric(all_cars['rate2'])
all_cars.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94352 entries, 0 to 104661
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            94352 non-null  int64  
 1   bodyType              94352 non-null  object 
 2   brand                 94352 non-null  object 
 3   car_url               94352 non-null  object 
 4   color                 94352 non-null  object 
 5   description           94352 non-null  object 
 6   engineDisplacement    94352 non-null  object 
 7   enginePower           94352 non-null  object 
 8   equipment_dict        94352 non-null  object 
 9   fuelType              94352 non-null  object 
 10  mileage               94352 non-null  float64
 11  modelDate             94352 non-null  float64
 12  model_info            94352 non-null  object 
 13  model_name            94352 non-null  object 
 14  numberOfDoors         94352 non-null  float64
 15  productionDate    

In [53]:
# удалим лишние признаки
all_cars.drop(['fuel_rate', 'contain2', 'contain2_split', 'rate2', 'Unnamed: 0'], axis=1, inplace=True)
all_cars.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94352 entries, 0 to 104661
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              94352 non-null  object 
 1   brand                 94352 non-null  object 
 2   car_url               94352 non-null  object 
 3   color                 94352 non-null  object 
 4   description           94352 non-null  object 
 5   engineDisplacement    94352 non-null  object 
 6   enginePower           94352 non-null  object 
 7   equipment_dict        94352 non-null  object 
 8   fuelType              94352 non-null  object 
 9   mileage               94352 non-null  float64
 10  modelDate             94352 non-null  float64
 11  model_info            94352 non-null  object 
 12  model_name            94352 non-null  object 
 13  numberOfDoors         94352 non-null  float64
 14  productionDate        94352 non-null  float64
 15  super_gen         

сохраним этот датасет для дальнейшей работы и проведения EDA, Feature Engineering и посторения моделей

In [54]:
all_cars.to_csv("dataset_V2.csv")